In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_core.output_parsers import StrOutputParser
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import re

# Define a URL base onde o serviço Ollama está rodando localmente
base_url = 'http://localhost:11434'

# Escolhe qual modelo de linguagem será usado
model = 'llama3.2:3b'

# Inicializa o cliente do modelo Ollama com a URL base e o modelo escolhido
llm = ChatOllama(base_url=base_url, model=model)

# Define um template de mensagem do sistema
system = SystemMessagePromptTemplate.from_template("""
Você é um assistente de IA útil que responde a perguntas de usuários relacionadas à análise de perfis do LinkedIn e faz comparações entre diferentes perfis. Você atribui pontuações e gera relatórios comparativos, de acordo com os critérios estabelecidos abaixo.

Critérios de Pontuação:
- **Recomendações do último ano (20 pontos)**
- **Recomendações há mais de 2 anos (10 pontos)**
- **Título no LinkedIn preenchido (10 pontos)**
- **Resumo no perfil (Sobre) (25 pontos)**
- **Experiência profissional detalhada (20 pontos)**
- **Experiência recente com destaque de conquistas (15 pontos)**
- **Certificações recentes (15 pontos)**
- **Cursos concluídos nos últimos 2 anos (10 pontos)**
- **Conexões de alto nível (10 pontos)**
- **Participação em grupos ou comunidades relevantes (10 pontos)**
- **Publicações e Atividade (15 pontos)**
- **Endossos de habilidades (15 pontos)**
- **Idiomas adicionais (10 pontos)**
- **Projetos ou portfólio (20 pontos)**

Compare os dois ou mais perfis do LinkedIn fornecidos, atribuindo pontos a cada critério e fornecendo uma análise comparativa detalhada. Seja objetivo e forneça uma pontuação total e sugestões de melhorias.
""")

# Função para fazer perguntas ao modelo de linguagem
def ask_llm(prompt):
    prompt = HumanMessagePromptTemplate.from_template(prompt)
    messages = [system, prompt]
    template = ChatPromptTemplate(messages)
    qna_chain = template | llm | StrOutputParser()
    return qna_chain.invoke({})

# Função para capturar e limpar o conteúdo de um perfil do LinkedIn usando Selenium
def get_linkedin_data(profile_url):
    # Abrir o Chrome
    driver = webdriver.Chrome()

    # Com o Chrome aberto, acessar o link de login do linkedin
    driver.get("https://www.linkedin.com/login")
    email = driver.find_element(By.ID, 'username')
    email.send_keys(os.getenv('EMAIL'))
    password = driver.find_element(By.ID, 'password')
    password.send_keys(os.getenv('PASSWORD'))
    password.submit()

    # Espera para garantir que o login seja realizado
    driver.implicitly_wait(5)

    driver.get(profile_url)

    # Captura o HTML da página
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Encontra a classe principal que mostra tudo do perfil
    profile = soup.find('main', {'class': 'AFoQvhgARpwZuvkYcojIRfUxjAGCtvmbnCCI'})

    # Trazer todas as sessões do perfil
    sections = profile.find_all('section', {'class': 'artdeco-card'})

    # Extrair texto de cada sessão
    sections_text = [section.get_text() for section in sections]

    # Limpar o texto extraído para remover duplicações e excessos
    def clean_text(text):
        text = re.sub(r'\n+', '\n', text)
        text = re.sub(r'\t+', '\t', text)
        text = re.sub(r'\t\s+', ' ', text)
        text = re.sub(r'\n\s+', '\n', text)
        return text

    sections_text = [clean_text(section) for section in sections_text]

    # Fechar o WebDriver após a coleta dos dados
    driver.quit()

    return sections_text

# Função para comparar dois perfis
def compare_profiles(profile_url_1, profile_url_2):
    # Captura os dados dos dois perfis
    profile_1_data = get_linkedin_data(profile_url_1)
    profile_2_data = get_linkedin_data(profile_url_2)

    # Organize os dados raspados de ambos os perfis em um formato adequado
    profile_data_1 = {
        "Nome e título": profile_1_data[0],
        "Atividades e publicações": profile_1_data[1],
        "Competências": profile_1_data[2],
        "Recomendações": profile_1_data[3]
    }

    profile_data_2 = {
        "Nome e título": profile_2_data[0],
        "Atividades e publicações": profile_2_data[1],
        "Competências": profile_2_data[2],
        "Recomendações": profile_2_data[3]
    }

    # Template para comparação dos perfis
    template = f"""
    Compare os seguintes perfis do LinkedIn, atribuindo pontuações de acordo com os critérios mencionados acima e forneça uma análise comparativa detalhada.

    ### Perfil 1 - Dados do LinkedIn:
    Nome e título: {profile_data_1["Nome e título"]}
    Atividades e publicações: {profile_data_1["Atividades e publicações"]}
    Competências: {profile_data_1["Competências"]}
    Recomendações: {profile_data_1["Recomendações"]}

    ### Perfil 2 - Dados do LinkedIn:
    Nome e título: {profile_data_2["Nome e título"]}
    Atividades e publicações: {profile_data_2["Atividades e publicações"]}
    Competências: {profile_data_2["Competências"]}
    Recomendações: {profile_data_2["Recomendações"]}

    ### Dados extraídos:

    1. Comparação de Recomendações: Qual perfil tem mais recomendações recentes? Qual tem mais relevância?
    2. Comparação de Títulos: Qual título está mais otimizado? Qual perfil tem o título mais relevante para recrutadores?
    3. Comparação de Resumos: Qual resumo está mais detalhado e atrativo? Qual perfil se destaca mais?
    4. Comparação de Experiência Profissional: Quem tem mais experiência e realizações significativas? Qual perfil se destaca?
    5. Comparação de Competências: Qual perfil tem uma lista mais forte de competências?
    6. Pontuação geral para cada perfil e sugestões de melhorias.

    Compare e forneça uma análise detalhada entre os dois perfis.
    """

    # Chama a função 'ask_llm' para comparar os perfis
    response = ask_llm(template)

    # Exibe a resposta do modelo
    print(response)

# Exemplo de URLs de perfis do LinkedIn
profile_url_1 = 'https://www.linkedin.com/in/sarahbatagioti/'
profile_url_2 = 'https://www.linkedin.com/in/marianne-val%C3%A9rio-nunes-701568292/'

# Compara os dois perfis
compare_profiles(profile_url_1, profile_url_2)


**Análise Comparativa**

### Comparação de Recomendações:

O Perfil 1 tem mais recomendações recentes, com um total de 2 recomendações há menos de 12 meses, enquanto o Perfil 2 não possui nenhuma recomendação recente.

A recomendação do Perfil 1 é mais relevante, pois é uma recomendação de um colega de trabalho e tem um histórico de colaboração no mesmo projeto. Já a recomendação do Perfil 2 parece ser menos relevante, pois não há informações sobre quem a fez ou por que.

Pontuação: Recomendações (Perfil 1: 20 pontos, Perfil 2: 0 pontos)

### Comparação de Títulos:

O título do Perfil 1 está mais otimizado, pois inclui todas as habilidades relevantes para um desenvolvedor full stack, como Python, JavaScript, HTML e CSS. Já o título do Perfil 2 é muito genérico e não oferece informações sobre as habilidades específicas.

Pontuação: Títulos (Perfil 1: 10 pontos, Perfil 2: 0 pontos)

### Comparação de Resumos:

O resumo do Perfil 1 está mais detalhado e atrativo, pois apresenta uma breve 